# Sklearn

In [1]:
from topic_modeling_functions import *
from tqdm import tqdm_notebook

pd.set_option('display.max_colwidth', -1)

In [2]:
start = 5  # min n_topics
stop = 21  # max n_topics + 1
step = 1
n_top_words = 10

In [3]:
count_vect = CountVectorizer(min_df=2, lowercase=False, tokenizer=lambda r: r)
tfidf_vect = TfidfVectorizer(min_df=2, lowercase=False, tokenizer=lambda r: r)

### Пример с текстами А. Волкова

In [4]:
df_volkov = pd.read_pickle('authors.pkl').iloc[0]

In [5]:
%%time

count_data, count_feature_names = get_data_vectorized(count_vect, df_volkov.text_pymystem_list)
tfidf_data, tfidf_feature_names = get_data_vectorized(tfidf_vect, df_volkov.text_pymystem_list)

CPU times: user 24.4 ms, sys: 6.46 ms, total: 30.9 ms
Wall time: 32.3 ms


In [6]:
%%time

# nmf падает, если n_topics > числа документов коллекции

lda_res = get_models_for_n_topics('lda', count_data, start, stop, step)
nmf_res = get_models_for_n_topics('nmf', tfidf_data, start, stop, step)

CPU times: user 1.18 s, sys: 68.6 ms, total: 1.25 s
Wall time: 1.15 s


In [7]:
%%time

lda_topics = get_n_top_words(lda_res, count_feature_names, n_top_words)
nmf_topics = get_n_top_words(nmf_res, tfidf_feature_names, n_top_words)

CPU times: user 20.9 ms, sys: 5.53 ms, total: 26.4 ms
Wall time: 13.9 ms


In [8]:
df_volkov_results = pd.DataFrame({'lda': lda_topics}, index=get_index(start, stop, step))
df_volkov_results['nmf'] = pd.Series(nmf_topics, index=get_index(start, tfidf_data.shape[0] + 1, step))

In [9]:
df_volkov_results.loc['n_topics_5']

,lda,nmf
n_topic_1,"[элли, девочка, железный, гудвин, путешественник, дровосек, лев, гингем, изумрудный, волшебница]","[страна, урфин, который, марран, город, волшебный, джюс, изумрудный, страшило, урфино]"
n_topic_2,"[страна, который, элли, урфин, волшебный, город, страшило, изумрудный, свой, помощь]","[элли, гудвин, девочка, бастинд, лев, дровосек, железный, путешественник, волшебница, фургон]"
n_topic_3,"[элли, который, страна, город, житель, гудвин, джюс, изумрудный, время, путешественник]","[генерал, энни, гном, тим, страшило, похищать, камень, это, совет, усыпительный]"
n_topic_4,"[страна, урфин, город, страшило, свой, помощь, знать, элли, ребенок, попадать]","[король, подземный, страна, время, ребенок, вода, элли, фред, который, тотошка]"
n_topic_5,"[страна, который, урфин, изумрудный, свой, домой, волшебный, сделать, джюс, дорога]","[колдунья, гном, чарли, книга, руф, билан, дракон, волшебный, страна, тилль]"


In [10]:
df_volkov_results.loc[('n_topics_6', 'n_topic_5')]

lda    [страна, который, урфин, изумрудный, домой, свой, сделать, джюс, волшебный, дерево]
nmf    [колдунья, гном, чарли, книга, руф, билан, дракон, волшебный, страна, тилль]       
Name: (n_topics_6, n_topic_5), dtype: object

### Для каждой литературной традиции:

In [11]:
df_traditions = pd.read_pickle('traditions_topic_modeling.pkl')

In [12]:
%%time

traditions_topics = {}

for i in tqdm_notebook(df_traditions.index):
    traditions_topics[df_traditions.loc[i, 'tradition']] = get_words_for_topics(df_traditions.loc[i,
                                                                                'text_pymystem_list'],
                                                                                count_vect, tfidf_vect,
                                                                                start, stop, step, n_top_words)

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))


CPU times: user 7min 22s, sys: 7min 49s, total: 15min 12s
Wall time: 5min 9s


In [13]:
traditions_topics.keys()

dict_keys(['Прочая европейская литература', 'Латиноамериканская литература', 'Американская литература', 'Прочая литература', 'Прочая восточная литература', 'Европейская литература', 'Античная литература', 'Русская литература', 'Скандинавская литература', 'Британская литература'])

In [14]:
traditions_topics['Европейская литература'].loc['n_topics_10']

,lda,nmf
n_topic_1,"[свой, который, весь, это, человек, время, становиться, отец, мочь, жизнь]","[свой, который, человек, весь, это, жизнь, становиться, время, друг, отец]"
n_topic_2,"[давид, брут, саул, мороз, теодоро, свой, цезарь, который, альваро, ионафан]","[дон, хуан, донья, луис, карлос, мануэль, родриго, жуан, педро, кихот]"
n_topic_3,"[фауст, мефистофель, магомет, сеид, ансельмо, свой, который, омар, шейх, весь]","[принц, принцесса, король, курфюрст, фернандо, эмилия, герцог, турандот, граф, дженнаро]"
n_topic_4,"[галилей, шляпа, бартоло, розин, свой, базиль, фигаро, который, весь, это]","[де, госпожа, вальмон, манон, жюльен, маркиза, эжени, мадемуазель, шевалье, г]"
n_topic_5,"[немо, фигаро, нед, лот, керубиный, гофман, свой, который, подводный, судно]","[маркиз, маркиза, флориндо, беатриче, кавалер, мария, панталоне, синьора, фабрицио, чекко]"
n_topic_6,"[свой, весь, который, становиться, это, время, друг, гленарывать, человек, отец]","[генрих, король, матильда, екатерина, наваррский, королева, марго, колокол, жанна, гиз]"
n_topic_7,"[свой, который, мочь, это, граф, становиться, весь, маркиз, шляпа, кавалер]","[эмма, мистер, дэвид, шарль, родольф, элтон, фрэнк, леон, лера, флора]"
n_topic_8,"[гастон, айртон, свой, гленарывать, который, дункан, грант, фокленд, поселенец, весь]","[цезарь, клеопатра, антиох, тит, брут, нерон, рим, царица, римский, император]"
n_topic_9,"[мара, свой, который, ткач, весь, человек, это, шейх, становиться, сын]","[граф, сюзанна, фигаро, графиня, керубиный, леон, розин, бартоло, базиль, марселина]"
n_topic_10,"[свой, сесиль, весь, который, жанна, леон, это, становиться, дом, время]","[робинзон, остров, испанец, корабль, пятница, дикарь, капитан, посев, леон, англия]"


### Для авторов (6 и более текстов в корпусе):

In [15]:
df_authors = pd.read_pickle('authors.pkl')

In [16]:
%%time

authors_topics = {}

for i in tqdm_notebook(df_authors.index):
    authors_topics[df_authors.loc[i, 'author']] = get_words_for_topics(df_authors.loc[i, 'text_pymystem_list'],
                                                                                count_vect, tfidf_vect,
                                                                                start, stop, step, n_top_words)

HBox(children=(IntProgress(value=0, max=6), HTML(value='')))


CPU times: user 6.83 s, sys: 3.09 s, total: 9.92 s
Wall time: 5.82 s


In [17]:
authors_topics.keys()

dict_keys(['Кальдерон', 'Расин', 'Корнель', 'Еврипид', 'Волков', 'Вольтер'])

In [18]:
authors_topics['Расин'].loc['n_topics_6']

,lda,nmf
n_topic_1,"[свой, закон, отец, сын, готовый, цезарь, гермиона, нерон, оставаться, который]","[гермиона, свой, убийство, сын, долг, убивать, говорить, приходить, решать, это]"
n_topic_2,"[ахилл, дочь, свой, агамемнон, царь, нерон, клитемнестр, мать, отец, мочь]","[цезарь, император, рим, закон, свой, народ, римский, мочь, решение, любовь]"
n_topic_3,"[готовый, народ, отец, храм, сообщать, страшный, случай, который, ребенок, нерон]","[ахилл, агамемнон, дочь, клитемнестр, свой, царь, троя, герой, алтарь, греческий]"
n_topic_4,"[нерон, свой, царь, который, сын, это, едва, хотеть, весь, агамемнон]","[храм, мальчик, царь, царство, наследник, готовый, свой, ребенок, родитель, который]"
n_topic_5,"[встреча, дочь, храм, свой, хотеть, воздавать, совершать, нерон, царь, греческий]","[сын, тесей, свой, отец, страсть, хотеть, говорить, власть, любовь, мочь]"
n_topic_6,"[свой, нерон, сын, хотеть, который, мочь, цезарь, говорить, весь, гермиона]","[нерон, император, мать, цезарь, свой, рим, дворец, это, который, ибо]"


### Для всей выборки:

In [19]:
df = pd.read_pickle('metatable_preprocessed.pkl')

In [20]:
%%time

df_results = get_words_for_topics(df.text_pymystem_list, count_vect, tfidf_vect, start, stop, step, n_top_words)

CPU times: user 8min 6s, sys: 9min 6s, total: 17min 13s
Wall time: 6min 23s


In [21]:
df_results.loc['n_topics_5']

,lda,nmf
n_topic_1,"[пеппи, степан, ленька, динка, лева, теркин, мишка, кролик, неля, ватанабэ]","[свой, который, человек, весь, это, жизнь, становиться, время, дом, отец]"
n_topic_2,"[человек, свой, это, который, мочь, ваш, компания, должный, весь, время]","[ваш, клиент, компания, продукт, пример, бизнес, сотрудник, человек, использовать, решение]"
n_topic_3,"[хуан, дон, уэсли, дэвис, цезарь, донья, мануэль, джонатан, волынский, тарелкин]","[генрих, король, матильда, королева, принцесса, наваррский, екатерина, марго, колокол, юдифь]"
n_topic_4,"[свой, который, весь, это, человек, время, становиться, дом, жизнь, год]","[иван, царь, иванович, васильевич, конь, баба, яга, жар, князь, кощей]"
n_topic_5,"[свой, который, весь, это, король, бог, царь, сын, становиться, отец]","[дон, хуан, донья, жуан, луис, карлос, родриго, мануэль, король, педро]"


# Gensim

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [32]:
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [23]:
%%time

# Build the bigram and trigram models
bigram = gensim.models.Phrases(df['text_pymystem_list'], 
                               min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(df['text_pymystem_list'], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

CPU times: user 54.8 s, sys: 279 ms, total: 55.1 s
Wall time: 55.1 s


In [24]:
trigram_mod[bigram_mod[df['text_pymystem_list'][2]]][0:20]

['лето',
 'последний',
 'главный_героиня',
 'аля',
 'амосов',
 'родной',
 'деревня',
 'летовка',
 'прошлый',
 'год',
 'похороны',
 'мать',
 'хотеть',
 'узнавать',
 'новость',
 'тетка',
 'анисья',
 'маня',
 'который',
 'приезжать']

In [25]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

### Create the Dictionary and Corpus needed for Topic Modeling

In [26]:
%%time

id2word = corpora.Dictionary(df['text_pymystem_list'])

# Create Corpus
data_words_bigrams = make_bigrams(df['text_pymystem_list'])

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_words_bigrams]

CPU times: user 12 s, sys: 125 ms, total: 12.1 s
Wall time: 12.1 s


In [27]:
corpus[:1]

[[(0, 13),
  (1, 22),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 5),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 3),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 4),
  (24, 3),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (30, 4),
  (31, 1),
  (32, 3),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 2),
  (39, 1),
  (40, 2),
  (41, 3),
  (42, 4),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 2),
  (47, 2),
  (48, 7),
  (49, 2),
  (50, 2),
  (51, 1),
  (52, 2),
  (53, 1),
  (54, 1),
  (55, 3),
  (56, 2),
  (57, 3),
  (58, 2),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 3),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 2),
  (69, 1),
  (70, 2),
  (71, 1),
  (72, 1),
  (73, 2),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 3),
  (78, 1),
  (80, 1),
  (82, 1),
  (83, 1),
  (84, 2),
  (85, 2),
  (86, 1),
  (87, 2),
  (88, 3),
  (89, 1),
  (90, 1),
  (91, 1),
  (92, 1),
  (93, 1),
  (94, 1),
  (95,

In [28]:
id2word[50]

'девушка'

In [29]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('анкудин', 13),
  ('анюта', 22),
  ('балалайка', 1),
  ('бежать', 1),
  ('безделье', 1),
  ('бесплатенмельник', 1),
  ('благодарить', 1),
  ('благословлять', 1),
  ('боярин', 1),
  ('брифли', 1),
  ('вдали', 1),
  ('велеть', 5),
  ('верить', 1),
  ('вернуться', 1),
  ('весть', 1),
  ('ветер', 1),
  ('видение', 1),
  ('видеть', 3),
  ('вино', 1),
  ('внешне', 1),
  ('возвращение', 1),
  ('вокруг', 1),
  ('вопрос', 4),
  ('ворожба', 3),
  ('ворожить', 1),
  ('ворота', 1),
  ('вперед', 1),
  ('вращать', 1),
  ('встречать', 4),
  ('вступать', 1),
  ('выбор', 3),
  ('вызывать', 1),
  ('выносить', 1),
  ('выпивать', 1),
  ('выпроваживать', 1),
  ('выражать', 1),
  ('вырастать', 2),
  ('выслушивать', 1),
  ('выходить', 2),
  ('глаз', 3),
  ('говорить', 4),
  ('гоняться', 1),
  ('гулять', 1),
  ('давать', 1),
  ('давно', 2),
  ('двор', 2),
  ('дворянин', 7),
  ('дворянский', 2),
  ('девушка', 2),
  ('действие', 1),
  ('делать', 2),
  ('дело', 1),
  ('деньги', 1),
  ('деревня', 3),
  ('дерев

### Building the Topic Model

In [30]:
%%time

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

CPU times: user 2min 57s, sys: 3min 37s, total: 6min 35s
Wall time: 2min 36s


### View the topics in LDA model

In [33]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.008*"увидеть" + 0.006*"собака" + 0.005*"зверь" + 0.005*"пес" + '
  '0.005*"бенвенуто" + 0.005*"вода" + 0.005*"корабль" + 0.005*"лес" + '
  '0.005*"становиться" + 0.005*"дерево"'),
 (1,
  '0.018*"который" + 0.016*"свой" + 0.009*"дом" + 0.008*"отец" + 0.008*"время" '
  '+ 0.008*"это" + 0.007*"весь" + 0.007*"становиться" + 0.007*"друг" + '
  '0.006*"узнавать"'),
 (2,
  '0.016*"война" + 0.013*"солдат" + 0.011*"немец" + 0.008*"армия" + '
  '0.008*"таня" + 0.008*"идти" + 0.007*"генерал" + 0.007*"офицер" + '
  '0.007*"бой" + 0.006*"коля"'),
 (3,
  '0.012*"бог" + 0.009*"это" + 0.009*"свой" + 0.007*"орест" + 0.007*"сын" + '
  '0.007*"убивать" + 0.006*"хор" + 0.005*"мать" + 0.005*"отец" + '
  '0.005*"агамемнон"'),
 (4,
  '0.033*"король" + 0.024*"парцифаль" + 0.014*"рыцарь" + 0.010*"юноша" + '
  '0.009*"замок" + 0.008*"королева" + 0.008*"который" + 0.008*"свой" + '
  '0.008*"анфортас" + 0.007*"отправляться"'),
 (5,
  '0.021*"гамлет" + 0.015*"антон" + 0.012*"мартин" + 0.012*"герш" + '
  

### Compute Model Perplexity and Coherence Score

In [34]:
%%time

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=df['text_pymystem_list'],
                                     dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.921506228284118

Coherence Score:  0.43487319356205256
CPU times: user 25.7 s, sys: 33.3 s, total: 59 s
Wall time: 2min 36s


### Visualize the topics-keywords

In [35]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      28.069025  1        1       0.269550 -0.017474
14     13.189507  1        2       0.239646 -0.084729
6      11.920007  1        3       0.226748 -0.070364
3      8.782454   1        4       0.212362  0.105406
15     7.686161   1        5       0.188534 -0.179187
8      4.786381   1        6       0.181551  0.130922
4      3.005141   1        7       0.091028  0.209543
0      2.846661   1        8      -0.023360 -0.011305
11     2.690516   1        9      -0.038823 -0.049670
17     2.552323   1        10     -0.063582 -0.159337
7      2.135294   1        11     -0.058126 -0.118795
2      2.102519   1        12     -0.102083 -0.000867
13     1.790023   1        13     -0.143035 -0.056929
12     1.748410   1        14     -0.077230 -0.144056
19     1.522004   1        15     -0.029529  0.227655
16     1.468196   1        16     -0.043703  0.140209
9      1.177337   1        17     -0.198053  0.018853
18     1.045083   1        18     -0.202663  0.036876
5      0.766915   1        19     -0.211636  0.009481
10     0.716028   1        20     -0.217596  0.013769, topic_info=      Category          Freq          Term         Total  loglift  logprob
term                                                                      
29384  Default  13733.000000  свой          13733.000000  30.0000  30.0000
6820   Default  3634.000000   дом           3634.000000   29.0000  29.0000
32586  Default  7359.000000   человек       7359.000000   28.0000  28.0000
31041  Default  1199.000000   нильс         1199.000000   27.0000  27.0000
11649  Default  2778.000000   жена          2778.000000   26.0000  26.0000
28968  Default  1267.000000   король        1267.000000   25.0000  25.0000
26269  Default  3855.000000   отец          3855.000000   24.0000  24.0000
13268  Default  1398.000000   герой         1398.000000   23.0000  23.0000
51323  Default  3225.000000   сын           3225.000000   22.0000  22.0000
38853  Default  2311.000000   девушка       2311.000000   21.0000  21.0000
50440  Default  2206.000000   дочь          2206.000000   20.0000  20.0000
16677  Default  11376.000000  который       11376.000000  19.0000  19.0000
34728  Default  966.000000    царь          966.000000    18.0000  18.0000
54005  Default  1396.000000   мальчик       1396.000000   17.0000  17.0000
51374  Default  3924.000000   жизнь         3924.000000   16.0000  16.0000
36357  Default  7915.000000   это           7915.000000   15.0000  15.0000
40057  Default  1423.000000   семья         1423.000000   14.0000  14.0000
38219  Default  890.000000    парцифаль     890.000000    13.0000  13.0000
55598  Default  1253.000000   бог           1253.000000   12.0000  12.0000
26342  Default  2614.000000   мать          2614.000000   11.0000  11.0000
5813   Default  1283.000000   война         1283.000000   10.0000  10.0000
24836  Default  1945.000000   молодой       1945.000000   9.0000   9.0000 
27332  Default  1634.000000   отправляться  1634.000000   8.0000   8.0000 
50767  Default  1699.000000   деньги        1699.000000   7.0000   7.0000 
15819  Default  2757.000000   узнавать      2757.000000   6.0000   6.0000 
57762  Default  3297.000000   год           3297.000000   5.0000   5.0000 
873    Default  4340.000000   мочь          4340.000000   4.0000   4.0000 
30994  Default  1717.000000   убивать       1717.000000   3.0000   3.0000 
8824   Default  2519.000000   говорить      2519.000000   2.0000   2.0000 
2253   Default  2042.000000   любовь        2042.000000   1.0000   1.0000 
...        ...          ...        ...              ...      ...      ... 
56150  Topic20  69.668556     плюм          70.582794     4.9262  -4.8264 
41159  Topic20  67.904167     иосел         68.818405     4.9258  -4.8520 
55662  Topic20  67.821724     итен          68.735962     4.9258  -4.8532 
21944  Topic20  61.333847     тарелкин      62.248085     4.9244  -4.

### Building LDA Mallet Model

In [ ]:
# проблема с Mallet: 
#                'CalledProcessError: Command '/mallet-2.0.8/bin/mallet import-file --preserve-case 
#                 --keep-sequence --remove-stopwords --token-regex "\S+" --input /tmp/84e43d_corpus.txt 
#                 --output /tmp/84e43d_corpus.mallet' returned non-zero exit status 127'

In [ ]:
%%time

# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = '/mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

### How to find the optimal number of topics for LDA?

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus,
                                                 num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
%%time

# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word,
                                                        corpus=corpus,
                                                        texts=df['text_pymystem_list'],
                                                        start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

### Finding the dominant topic in each sentence

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic, 4),
                                                                  topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

### Find the most representative document for each topic

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

### Topic distribution across documents

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics